In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType, TimestampType
from dotenv import load_dotenv
import os
import json
import time
import boto3
from pyspark.sql import SparkSession
from datetime import datetime 

In [2]:
# Load environment variables from .env file
load_dotenv()

# Initialize AWS credentials from the .env file
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
AWS_REGION = os.getenv('AWS_REGION')

In [3]:
# from pyspark.sql import SparkSession

# # Path to your local JAR files
# local_jars = "/Users/borja/Documents/Somniumrema/projects/de/route_optimizer/jars/aws-java-sdk-kinesis-1.12.364.jar"

# # Initialize Spark session with Delta and S3 settings
# spark = SparkSession.builder \
#     .appName("KinesisToDeltaLake") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0,org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.1026") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY_ID) \
#     .config("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_ACCESS_KEY) \
#     .config("spark.hadoop.fs.s3a.session.token", AWS_SESSION_TOKEN) \
#     .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
#     .config("spark.sql.files.maxPartitionBytes", "134217728") \
#     .config("spark.driver.memory", "4g") \
#     .config("spark.executor.memory", "4g") \
#     .getOrCreate()

# # Optional: Adjust logging level
# spark.sparkContext.setLogLevel("WARN")

In [4]:
import boto3

# Initialize boto3 client for Glue
glue_client = boto3.client(
    'glue',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,  # Only necessary if using temporary credentials
    region_name=AWS_REGION
)

# Verify if the job exists
job_name = 'kinesis-to-dispatcher'

In [7]:
# List all Glue sessions
response = glue_client.list_sessions()

# Print all session IDs
for session in response['Sessions']:
    print(f"Session ID: {session['Id']}")

In [9]:
import boto3



# Start a Glue interactive session
response = glue_client.create_session(
    Id='trying',
    Role='arn:aws:iam::277709186030:role/LabRole',  # Replace with your Glue service role
    WorkerType='G.1X',  # Or use 'G.2X' for larger workers
    Command={
        'Name': 'glueetl'  # Must be 'glueetl' for interactive sessions
    },
    NumberOfWorkers=2
)

# Print session details
session_id = response['Session']['Id']
print(f"Session started: {session_id}")

Session started: trying


In [21]:
# Stop the Glue session
glue_client.delete_session(Id=session_id)
print(f"Session {session_id} stopped.")

Session trying stopped.


In [18]:
# Import required AWS Glue and Spark libraries
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
import sys

# Initialize the SparkContext and GlueContext
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

ModuleNotFoundError: No module named 'awsglue'

In [16]:
import boto3
from pyspark.sql import SparkSession

# Kinesis Stream configurations
kinesis_stream_name = "OrderStreamForDispatching"  # Replace with your Kinesis stream name



# Step 1: Initialize the Spark session with the Kinesis connector package
spark = SparkSession.builder \
    .appName("KinesisToDeltaLake") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kinesis_2.12:3.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Step 3: Read from Kinesis stream using Spark
kinesis_df = spark \
    .readStream \
    .format("kinesis") \
    .option("streamName", kinesis_stream_name) \
    .option("region", AWS_REGION) \
    .option("initialPosition", "LATEST") \
    .option("endpointUrl", f"https://kinesis.{AWS_REGION}.amazonaws.com") \
    .load()

# Show the DataFrame schema
kinesis_df.printSchema()

# Example transformation: filter the stream for specific data
transformed_df = kinesis_df.filter(kinesis_df["status"] == "READY_FOR_DISPATCH")

# Write the transformed stream to the console (for testing)
query = transformed_df \
    .writeStream \
    .format("console") \
    .start()

query.awaitTermination()



24/10/09 00:34:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Py4JJavaError: An error occurred while calling o64.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: kinesis. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:738)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.streaming.DataStreamReader.loadInternal(DataStreamReader.scala:157)
	at org.apache.spark.sql.streaming.DataStreamReader.load(DataStreamReader.scala:144)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.r

In [10]:
# Initialize boto3 client for Kinesis with your credentials
kinesis_client = boto3.client(
    'kinesis',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
    region_name=AWS_REGION
)

In [11]:
# Define the schema
schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("total_weight", DoubleType(), True),
    StructField("total_volume", DoubleType(), True),
    StructField("total_price", DoubleType(), True),
    StructField("order_timestamp", TimestampType(), True),
    StructField("status", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lon", DoubleType(), True)
])

In [12]:
# Function to convert the timestamps to proper datetime objects
def convert_timestamps(orders):
    for order in orders:
        if isinstance(order['order_timestamp'], str):
            # Only convert if the timestamp is a string
            order['order_timestamp'] = datetime.strptime(order['order_timestamp'], '%Y-%m-%d %H:%M:%S')
    return orders

# Function to convert the location field from MapType to StructType
def transform_location(order):
    if isinstance(order['location'], dict):
        location_data = order['location']
        order['location'] = {
            "address": location_data['address'],
            "lat": float(location_data['lat']),
            "lon": float(location_data['lon'])
        }
    return order

# Function to get the shard iterator
def get_shard_iterator(stream_name, shard_id):
    response = kinesis_client.get_shard_iterator(
        StreamName=stream_name,
        ShardId=shard_id,
        ShardIteratorType='LATEST'  # or 'LATEST' for new records
    )
    return response['ShardIterator']

# Function to read records from the Kinesis stream
def read_kinesis_records(stream_name, shard_iterator):
    while True:
        response = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
        records = response['Records']
        for record in records:
            # No need to base64 decode, just parse the data directly
            order_data = record['Data']
            order = json.loads(order_data)
            print("Received order:", order)

        # Update the shard iterator for the next batch of records
        shard_iterator = response['NextShardIterator']

        # Sleep to avoid hitting API rate limits
        time.sleep(1)

In [ ]:
import time
import json

# Initialize buffer, limits, and thresholds
order_buffer = []
weight_threshold = 700  # Example weight threshold (can be adjusted)
volume_threshold = 500  # Example volume threshold (can be adjusted)
time_threshold = 60 * 45 # Example time threshold (e.g., 30 minutes)
buffer_limit = 10000      # Optional, max number of orders in the buffer
last_flush_time = time.time()

# S3 path for Delta table
delta_table_path = "s3a://orders-for-dispatch/dispatching"

# Dispatcher function to read Kinesis records, accumulate, and process them
def dispatcher(shard_iterator):
    global order_buffer, last_flush_time

    while True:
        # Fetch records from Kinesis using shard_iterator
        response = kinesis_client.get_records(ShardIterator=shard_iterator)
        records = response['Records']

        # Parse and accumulate orders
        for record in records:
            order_data = json.loads(record['Data'])
            order_buffer.append(order_data)
            print(f"Accumulated order: {order_data}")

        # Convert timestamps if necessary
        order_buffer = convert_timestamps(order_buffer)

        # Calculate accumulated weight and volume
        total_weight = sum(order['total_weight'] for order in order_buffer)
        total_volume = sum(order['total_volume'] for order in order_buffer)
        time_elapsed = time.time() - last_flush_time

        # Check if any thresholds are met (weight, volume, or time)
        if total_weight >= weight_threshold or total_volume >= volume_threshold or time_elapsed >= time_threshold:
            print(f"Threshold met: Dispatching {len(order_buffer)} orders.")
            
            # Update the status of all orders to 'DISPATCH_READY'
            for order in order_buffer:
                order['status'] = 'READY_FOR_DISPATCH'

            # Convert buffer to DataFrame and write to Delta table
            df = spark.createDataFrame(order_buffer, schema=schema)
            df.write.format("delta").mode("append").save(delta_table_path)
            print(f"Saved {len(order_buffer)} orders to Delta table.")
            
            # Clear the buffer and reset flush time
            order_buffer.clear()
            last_flush_time = time.time()

        # Update shard iterator for the next batch
        shard_iterator = response['NextShardIterator']
        time.sleep(2)  # Sleep to avoid rate limits

# Get the shard iterator
stream_name = 'OrderStreamForDispatching'  # Replace with your stream name
shard_id = 'shardId-000000000000'  # Get the shard ID from the stream's details
shard_iterator = get_shard_iterator(stream_name, shard_id)

# Start the dispatcher
dispatcher(shard_iterator)


In [9]:
spark.stop()